In [1]:
from gurobipy import *  # import the optimize solver Gurobi

In [2]:
number_of_product = 2 # Set the index for number of juice product(1, 2)
number_of_machine = 2 # Set the index for number of machine(1, 2)
m = Model() # Import and create the model

Using license file C:\Users\chaoz\gurobi_licence\gurobi.lic
Academic license - for non-commercial use only


In [3]:
# Set the input Parameter: 
machine_hour = [[0.7, 0.75], [0.8, 0.9]] # Machine hours required to produce each type of juice on each machine
labor_hour = [[0.75, 0.75], [1.2, 1.0]]  # Labor hours required to produce each type of juice on each machine  
unit_cost = [[16.5, 13.5], [18, 14.5]]   # Unit cost to produce each type of juice on each machine 
demand = [250, 170]  # Demand to produce each type of juice
each_machine = [200, 200]   # Labor hour available on each machine


In [4]:
# Set the Variable list: Number of each type of juice produced on each machine
# Set the variable nx to integer number
nx = [[0, 0], [0,0]]
for i in range(number_of_machine):
    for j in range(number_of_product):
        nx[i][j] = m.addVar(vtype=GRB.INTEGER, name='nx{}'.format(str(i + 1) + str(j + 1)))

In [5]:
# Set the Minimize Obijective: Net Cost
m.setObjective(quicksum([sum([unit_cost[i][j] * nx[i][j] for j in range(number_of_product)]) for i in range(number_of_machine)]) ,  GRB.MINIMIZE)

In [6]:
# Set Non Negative decision variable
for i in range(number_of_machine):
     for j in range(number_of_product):
            c1 = m.addConstr(nx[i][j]  >= 0)


In [7]:
# Machine hours constraint
c2 = []
for i in range(number_of_machine):
    c2.append(m.addConstr(sum([machine_hour[i][j]*nx[i][j] for j in range(number_of_product)]) <= each_machine[i]))

In [8]:
# Labor hours constraint
c3 = m.addConstr( quicksum([sum([labor_hour[i][j]*nx[i][j] for j in range(number_of_product)]) for i in range(number_of_machine)]) <= 500)
c7 = m.addConstr( quicksum([sum([labor_hour[i][j]*nx[i][j] for j in range(number_of_product)]) for i in range(number_of_machine)]) >= 380)

In [9]:
# Production Demand constraint
c4 = []
for j in range(number_of_product):
    c4.append(m.addConstr(sum([nx[i][j] for i in range(number_of_machine)]) >= demand[j]))

In [10]:
# Demand to produce each type of juice on each machine(at least half of the product 1 requirement must be produced on machine 2, and at least half of the product 2 requirement must be produced on machine 1)
c5 = m.addConstr(nx[0][1] >= 1/2 * demand[1])
c6 = m.addConstr(nx[1][0] >= 1/2 * demand[0])

In [11]:
# Run the optimize solver
m.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 12 rows, 4 columns and 22 nonzeros
Model fingerprint: 0xbf13db20
Variable types: 0 continuous, 4 integer (0 binary)
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [1e+01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+01, 5e+02]
Presolve removed 6 rows and 0 columns
Presolve time: 0.24s
Presolved: 6 rows, 4 columns, 16 nonzeros
Variable types: 0 continuous, 4 integer (0 binary)
Found heuristic solution: objective 6793.5000000

Root relaxation: objective 6.637308e+03, 3 iterations, 0.05 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 6637.30769    0    4 6793.50000 6637.30769  2.30%     -    0s
H    0     0                    6638.0000000 6637.30769  0.01%     -    0s

Explored 1 nodes (3 simplex iterations) in 0.48 seconds
Thread count was 8

In [12]:
# Get the Optimal Solution for X
m.printAttr('X')


    Variable            X 
-------------------------
        nx11          108 
        nx12          165 
        nx21          142 
        nx22            5 


In [13]:
# Get the Optimal Objective Value
m.ObjVal

6638.0